# Using Redis and OpenAI to chat with PDF documents

This notebook demonstrates how to use RedisAI and (Azure) OpenAI to chat with PDF documents. The PDF included is
a informational brochure about the Chevy Colorado pickup truck.

In this notebook, we will use LLamaIndex to chunk, vectorize, and store the PDF document in Redis as vectors
alongside associated text. The query interface provided by LLamaIndex will be used to search for relevant
information given queries from the user.

In [4]:
# Install the Python requirements
%pip install -r requirements.txt --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.0.1
    Uninstalling openai-1.0.1:
      Successfully uninstalled openai-1.0.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import sys
import logging

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
) # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import textwrap
import openai

from langchain.llms import AzureOpenAI, OpenAI
from langchain.embeddings import OpenAIEmbeddings
from llama_index.vector_stores import RedisVectorStore
from llama_index import LangchainEmbedding
from llama_index import (
    GPTVectorStoreIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    PromptHelper,
    ServiceContext,
    StorageContext
)

# Azure OpenAI | OpenAI (direct)

The notebook allows the user two choose between using the OpenAI and Azure OpenAI endpoints. Make sure to follow the instructions in the README and set the .env correctly according to whichever API you are using. 

NOTE: ONLY ONE API CAN BE USED AT A TIME.

- **[Use Azure OpenAI](#Azure-OpenAI)**
- **[Use OpenAI](#OpenAI) (direct)**

## Azure OpenAI 

Here we setup the AzureOpenAI models and API keys that we set by reading from the environment above. The ``PromptHelper`` sets the parameters for the OpenAI model. The classes defined here are used together to provide a QnA interface between the user and the LLM.

In [2]:
# setup Llama Index to use Azure OpenAI
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")


# Get the OpenAI model names ex. "text-embedding-ada-002"
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
text_model = os.getenv("OPENAI_TEXT_MODEL")
# get the Azure Deployment name for the model
embedding_model_deployment = os.getenv("AZURE_EMBED_MODEL_DEPLOYMENT_NAME")
text_model_deployment = os.getenv("AZURE_TEXT_MODEL_DEPLOYMENT_NAME")

print(f"Using OpenAI models: {embedding_model} and {text_model}")
print(f"Using Azure deployments: {embedding_model_deployment} and {text_model_deployment}")


Using OpenAI models: text-embedding-ada-002 and gpt-35-turbo
Using Azure deployments: <your deployment name here> and <your deployment name here>


In [5]:

llm = AzureOpenAI(deployment_name=text_model_deployment, model_kwargs={
    "api_key": openai.api_key,
    "api_base": openai.api_base,
    "api_type": openai.api_type,
    "api_version": openai.api_version,
})
llm_predictor = LLMPredictor(llm=llm)

embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model=embedding_model,
        deployment=embedding_model_deployment,
        openai_api_key= openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
        openai_api_version=openai.api_version,
    ),
    embed_batch_size=1,
)

## OpenAI

The ``OpenAI`` class provides a simple interface to the OpenAI API.


In [3]:
# setup Llama Index to use OpenAI direct
openai.api_type = "openai"
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_key = os.getenv("OPENAI_API_KEY")

# Get the OpenAI model names ex. "text-embedding-ada-002"
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
text_model = os.getenv("OPENAI_TEXT_MODEL")

print(f"Using OpenAI models: {embedding_model} and {text_model}")

Using OpenAI models: text-embedding-ada-002 and gpt-35-turbo


In [4]:
# Set up LLM
llm = OpenAI(model_kwargs={
    "api_key": openai.api_key,
    "api_base": openai.api_base,
    "api_type": openai.api_type,
    "api_version" : openai.api_version,
})
llm_predictor = LLMPredictor(llm=llm)

# Set up Embedding model
embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model=embedding_model,
        openai_api_version=openai.api_version,
        openai_api_key= openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
    ),
    embed_batch_size=1,
)

### LLamaIndex

[LlamaIndex](https://github.com/jerryjliu/llama_index) (GPT Index) is a project that provides a central interface to connect your LLM's with external data sources. It provides a simple interface to vectorize and store embeddings in Redis, create search indices using Redis, and perform vector search to find context for generative models like GPT.

Here we will use it to load in the documents (Chevy Colorado Brochure).

In [5]:
# load documents
documents = SimpleDirectoryReader('./docs').load_data()
print('Document ID:', documents[0].doc_id)

Document ID: c325c82f-9408-42fe-9a5a-c58e196ef5a3


Llamaindex also works with frameworks like langchain to make prompting and other aspects of a chat based application easier. Here we can use the ``PromptHelper`` class to help us generate prompts for the (Azure) OpenAI model. The will be off by default as it can be tricky to setup correctly.

In [6]:
# set number of output tokens
num_output = int(os.getenv("OPENAI_MAX_TOKENS"))
# max LLM token input size
max_input_size = int(os.getenv("CHUNK_SIZE"))
# set maximum chunk overlap
max_chunk_overlap = float(os.getenv("CHUNK_OVERLAP"))

prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

In [7]:
# define the service we will use to answer questions
# if you executive the Azure OpenAI code above, your Azure Models and creds will be used and the same for OpenAI
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embedding_llm,
    prompt_helper=prompt_helper # uncomment to use prompt_helper.
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Initialize Redis as a Vector Database

Now we have our documents read in, we can initialize the ``RedisVectorStore``. This will allow us to store our vectors in Redis and create an index.

The ``GPTVectorStoreIndex`` will then create the embeddings from the text chunks by calling out to OpenAI's API. The embeddings will be stored in Redis and an index will be created.

NOTE: If you didn't set the ``OPENAI_API_KEY`` environment variable, you will get an error here.

In [8]:
def format_redis_conn_from_env(using_ssl=False):
    start = "rediss://" if using_ssl else "redis://"
    # if using RBAC
    password = os.getenv("REDIS_PASSWORD", None)
    username = os.getenv("REDIS_USERNAME", "")
    if password != None:
        start += f"{username}:{password}@"

    return start + f"{os.getenv('REDIS_HOST')}:{os.getenv('REDIS_PORT')}"

# make using_ssl=True to use SSL with ACRE
redis_url = format_redis_conn_from_env(using_ssl=False)
print(f"Using Redis address: {redis_url}")


Using Redis address: redis://:0kYQxoG2RfkL0zVzQRXm0MSwBmVJ3D89+csfCH8E4fo=@sgn-llm.eastus2.redisenterprise.cache.azure.net:10000


In [9]:
# Create VectorStore
vector_store = RedisVectorStore(
    index_name="chevy_docs",
    index_prefix="blog",
    redis_url=redis_url,
    overwrite=True
)

# access the underlying client in the RedisVectorStore implementation to ping the redis instance
vector_store.client.ping()

True

In [10]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context
)

INFO:llama_index.vector_stores.redis:Creating index chevy_docs
Creating index chevy_docs
INFO:llama_index.vector_stores.redis:Added 27 documents to index chevy_docs
Added 27 documents to index chevy_docs


## Test the RAG pipeline!

Now that we have our document stored in the index, we can ask questions against the index. The index will use the data stored in itself as the knowledge base for the LLM.

In [11]:
query_engine = index.as_query_engine()
response = query_engine.query("What types of variants are available for the Chevrolet Colorado?")
print("\n", textwrap.fill(str(response), 100))

INFO:llama_index.vector_stores.redis:Using filters: *
Using filters: *
INFO:llama_index.vector_stores.redis:Querying index chevy_docs
Querying index chevy_docs
INFO:llama_index.vector_stores.redis:Found 2 results for query with id ['blog/vector_bd662262-87bf-4428-9e3e-155d20e99622', 'blog/vector_ef90fb76-37d3-40d6-b059-ce31151a1071']
Found 2 results for query with id ['blog/vector_bd662262-87bf-4428-9e3e-155d20e99622', 'blog/vector_ef90fb76-37d3-40d6-b059-ce31151a1071']

  The Chevrolet Colorado is available in Extended Cab, Crew Cab Short Box, and Crew Cab Long Box
variants. It includes a 2.5L DOHC 4-cylinder with Variable Valve Timing (VVT) and Direct Injection,
and a 3.6L DOHC V6 with Variable Valve Timing (VVT) and Direct Injection (Crew Cab 4x4 and Crew Cab
Long Box 2WD models). The transmission is either a 6-speed automatic, electronically controlled with
overdrive or an 8-speed automatic, electronically controlled with overdrive, Tow/Haul mode and Hitch
Guidance 1 (Crew Cab 4x4 

In [12]:
response = query_engine.query("What is the maximum towing capacity of the chevy colorado?")
print("\n", textwrap.fill(str(response), 100))

INFO:llama_index.vector_stores.redis:Using filters: *
Using filters: *
INFO:llama_index.vector_stores.redis:Querying index chevy_docs
Querying index chevy_docs
INFO:llama_index.vector_stores.redis:Found 2 results for query with id ['blog/vector_efcf4237-6811-4d27-b6b3-49f0062880f9', 'blog/vector_ef90fb76-37d3-40d6-b059-ce31151a1071']
Found 2 results for query with id ['blog/vector_efcf4237-6811-4d27-b6b3-49f0062880f9', 'blog/vector_ef90fb76-37d3-40d6-b059-ce31151a1071']

  The Chevy Colorado's maximum towing capacity is 7,700 lbs with the available GM-exclusive Duramax®
2.8L Turbo-Diesel engine and the Trailering Package, LT Convenience Package and Safety Package. The
Colorado Crew Cab ZR2 also features an AEV-designed aluminum wheels, bumper with winch provisions,
rear bumper with recovery points, five hot-stamped boron steel skid plates, fender flares, fog
lamps, front and rear floor liners with AEV logo, and AEV embroidered head restraints. However, due
to current supply chain short

In [13]:
response = query_engine.query("What are the main differences between the three engine types available for the Chevy Colorado?")
print("\n", textwrap.fill(str(response), 100))

INFO:llama_index.vector_stores.redis:Using filters: *
Using filters: *
INFO:llama_index.vector_stores.redis:Querying index chevy_docs
Querying index chevy_docs
INFO:llama_index.vector_stores.redis:Found 2 results for query with id ['blog/vector_a8ee72a5-fbef-4cea-acc7-633809a5777c', 'blog/vector_ef90fb76-37d3-40d6-b059-ce31151a1071']
Found 2 results for query with id ['blog/vector_a8ee72a5-fbef-4cea-acc7-633809a5777c', 'blog/vector_ef90fb76-37d3-40d6-b059-ce31151a1071']

  The main differences between the three engine types available for the Chevy Colorado are the power
output, torque output, displacement, bore and stroke, compression ratio, block and cylinder head
material, valvetrain, fuel delivery, maximum payload rating, maximum trailering weight rating, and
estimated fuel economy. The 2.5L DOHC I-4 engine has 200 hp @ 6300 rpm and 191 lb.-ft. of torque @
4400 rpm with a displacement of 2460 cc (150 cu. in.), a bore and stroke of 88 mm x 101 mm (3.46 in.
x 3.97 in.), a compression 